In [75]:
import pandas as pd
import numpy as np
import networkx as nx
import stellargraph as sg
from bmdcluster import blockdiagonalBMD

In [125]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader

transform = T.ToUndirected()  # Add reverse edge types.
data = OGB_MAG(root='/mnt/c/temp/working/data', preprocess='metapath2vec', transform=transform)[0]

train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 2,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=128,
    input_nodes=('paper', data['paper'].train_mask),
)

batch = next(iter(train_loader))

RuntimeError: No such operator torch_sparse::hetero_neighbor_sample

In [96]:
# for key, x in data.x_dict.items() :
# #     print(key,len(x), len(x[0]), sum(x[0]), x[0])
#     if key == 'author':
#         print(key, type(key))
#         print(x[0].shape, len(x[0]), sum(x[0]), min(x[0]), max(x[0]))
X = data.x_dict['author'].numpy()
# X = data.x_dict['paper'].numpy()
type(X), X.shape

(numpy.ndarray, (4057, 334))

In [107]:
K = 10
from bmdcluster import generalBMD
model = blockdiagonalBMD(n_clusters=K, use_bootstrap=True, b=5)
# model = generalBMD(n_clusters=K, B_ident=True, use_bootstrap=True, b=5)
model.fit(X, verbose=True)
data_labels = model.get_data_labels()
# cost, data_clusters, feature_clusters = model.fit_transform(data, verbose=True)
for k in range(K):
    print(sum(data_labels==k))

Iteration: 0 ............. Cost: 217.276
Iteration: 1 ............. Cost: 216.409
Iteration: 2 ............. Cost: 216.180
Iteration: 3 ............. Cost: 216.146
Iteration: 4 ............. Cost: 215.905
Iteration: 5 ............. Cost: 215.738
Iteration: 6 ............. Cost: 215.669
Iteration: 7 ............. Cost: 215.620
Iteration: 8 ............. Cost: 215.504
Iteration: 9 ............. Cost: 215.453
Iteration: 10 ............. Cost: 215.421
Convergence reached after 12 iterations
167
68
14
3
987
2818
0
0
0
0


In [120]:
# mask = data['author'].test_mask
# Y = X[mask]
# Y.shape
data.metadata()

(['author', 'paper', 'term', 'conference'],
 [('author', 'to', 'paper'),
  ('paper', 'to', 'author'),
  ('paper', 'to', 'term'),
  ('paper', 'to', 'conference'),
  ('term', 'to', 'paper'),
  ('conference', 'to', 'paper')])

In [115]:
from sklearn.cluster import KMeans
K = 5
k_means = KMeans(n_clusters=K, random_state=0).fit(X)
for k in range(K):
    print(sum(k_means.labels_==k))


249
34
2592
799
383


In [114]:
# from scipy.cluster.vq import kmeans,vq
# centroids, _ = kmeans(X,K)
# data_labels,_ = vq(X,centroids)
# for k in range(K):
#     print(sum(data_labels==k))

1616
372
145
174
335
328
21
123
590
353


In [111]:
idx

2.8642297

In [ ]:
# from popc import popc
# data_labels = popc(X)
# for k in range(K):
#     print(sum(data_labels==k))

In [2]:
# data.metadata()[1]
# data.x_dict['author'][0]#, data.edge_index_dict
# for key, x in data.x_dict.items() :
# #     print(key,len(x), len(x[0]), sum(x[0]), x[0])
#     print(key, type(key), key[0])
#     print(len(x[0]), min(x[0]), max(x[0]))
#     print(len(x[1]), min(x[1]), max(x[1]))
#     break
#     data.x_dict
# data.edge_index_dict

for key, x in data.edge_index_dict.items() :
    print(key, type(key), key[0])
    print(len(x[0]), min(x[0]), max(x[0]))
    print(len(x[1]), min(x[1]), max(x[1]))
    print(x)
    break
# print(model)

('author', 'to', 'paper') <class 'tuple'> author
19645 tensor(0) tensor(4056)
19645 tensor(0) tensor(14327)
tensor([[    0,     0,     1,  ...,  4054,  4055,  4056],
        [ 2364,  6457,  2365,  ..., 13891, 13891, 13892]])


In [4]:
# شماره نودهای عددی
for key, x in data.edge_index_dict.items() :
    num_authors = max(x[0]).item()+1
    papers_indices = [p.item()+num_authors for p in x[1]]
    edges_df = pd.DataFrame(
        {"source": x[0], "target": papers_indices }
    )
    break
edges_df["label"] = "writes"

In [12]:
"a-"+str(2)

'a-2'

In [56]:
# شماره نود با پیشوند
for key, x in data.edge_index_dict.items() :
    authors = ["a-"+str(p.item()) for p in x[0]]
    papers = ["p-"+str(p.item()) for p in x[1]]
    edges_df = pd.DataFrame(
        {"source": authors, "target": papers}
    )
    
    authors = ["a-"+str(p.item()) for p in np.unique(x[0])]
    papers = ["p-"+str(p.item()) for p in np.unique(x[1])]
    a_df = pd.DataFrame(index = authors)
    p_df = pd.DataFrame(index =  papers)
    break
# edges_df["label"] = "writes"

In [38]:
# a_df
# len(authors)

In [57]:
a_p_g = sg.StellarGraph({"authors": a_df, "papers": p_df}, edges_df)
print(a_p_g.info())

StellarGraph: Undirected multigraph
 Nodes: 18385, Edges: 19645

 Node types:
  papers: [14328]
    Features: none
    Edge types: papers-default->authors
  authors: [4057]
    Features: none
    Edge types: authors-default->papers

 Edge types:
    authors-default->papers: [19645]
        Weights: all 1 (default)
        Features: none


In [65]:
# type(x[0]), len(x[0])
# x = x[x!=value]
# edges_df
# num_authors.item(), type(num_authors)
# list(edges_df.loc[edges_df["source"]=="a-0"].target)
selected_authors = ["a-"+str(x) for x in range(10)]
selected_authors_papers = list(edges_df.loc[edges_df["source"].isin(selected_authors)].target)
sub_g = a_p_g.subgraph(selected_authors + list(set(selected_authors_papers)))
print(sub_g.info())

StellarGraph: Undirected multigraph
 Nodes: 78, Edges: 69

 Node types:
  papers: [68]
    Features: none
    Edge types: papers-default->authors
  authors: [10]
    Features: none
    Edge types: authors-default->papers

 Edge types:
    authors-default->papers: [69]
        Weights: all 1 (default)
        Features: none


In [61]:
G_all_nx = nx.from_pandas_edgelist(edges_df)#, edge_attr="label")
# nx.set_node_attributes(G_all_nx, "paper", "label")

In [62]:
G_all = sg.StellarGraph.from_networkx(G_all_nx)
print(G_all.info())

StellarGraph: Undirected multigraph
 Nodes: 18385, Edges: 19645

 Node types:
  default: [18385]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [19645]
        Weights: all 1 (default)
        Features: none


In [ ]:
# شاید بهتر باشه به جای زیرگراف، یک ماسک برای اتصالات انتخابی داشته باشیم
# چون در ادامه ماسکهای آموزش و اعتبارسنجی و تست رو داریم. 
# هر نودی که نمی خواهیم در آموزش مشارکت داشته باشه رو میشه در ماسک آموزش لحاظ کرد
# def sub_g(edge_index_dict, d_key, indices):
# #     d_key: drop key
# #     t_key: tuple key
#     for t_key, x in edge_index_dict.items():
#         if d_key in t_key:
            
            


In [ ]:
import numpy as np
with np.load('/mnt/c/temp/working/data/DBLP/raw/adjM.npz') as M:
    print(M['indices'])
    print(M['indptr'])
    print(M['format'])
    print(M['shape'])
    print(M['data'])

In [ ]:
M.files

In [121]:
%%time
%run test_01.py

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)
Epoch: 001, Loss: 1.3775, Train: 0.5575, Val: 0.4275, Test: 0.4965
Epoch: 002, Loss: 1.2896, Train: 0.5875, Val: 0.4650, Test: 0.5327
Epoch: 003, Loss: 1.1828, Train: 0.7550, Val: 0.5525, Test: 0.6334
Epoch: 004, Loss: 1.0495, Train: 0.8175, Val: 0.6275, Test: 0.6798
Epoch: 005, Loss: 0.8928, Train: 0.8850, Val: 0.6675, Test: 0.7123
Epoch: 006, Loss: 0.7271, Train: 0.9125, Val: 0.6875, Test: 0.7464
Epoch: 007, Loss: 0.5652, Train: 0.9525, Val: 0.7125, Test: 0.7660
Epoch: 008, Loss: 0.4

In [1]:
%run test_02.py

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)
Epoch: 001, Loss: 1.3807, Train: 0.5081, Val: 0.4425, Test: 0.5100
Epoch: 002, Loss: 1.3149, Train: 0.5867, Val: 0.4925, Test: 0.5375
Epoch: 003, Loss: 1.2252, Train: 0.7525, Val: 0.6625, Test: 0.7400
Epoch: 004, Loss: 1.1170, Train: 0.8106, Val: 0.7450, Test: 0.7900
Epoch: 005, Loss: 0.9861, Train: 0.8465, Val: 0.7800, Test: 0.8150
Epoch: 006, Loss: 0.8430, Train: 0.8618, Val: 0.8025, Test: 0.8200
Epoch: 007, Loss: 0.7004, Train: 0.8784, Val: 0.8225, Test: 0.8250
Epoch: 008, Loss: 0.5